In [ ]:
from import_libraries.project_functions import *

warnings.filterwarnings("ignore")

# data from MT5

In [ ]:
# symbol = 'EURUSD.pro'
data = get_historical_data() # D1
# data_m5 = get_historical_data(timeframe=mt5.TIMEFRAME_M5, symbol="GER30", start='2022-01-01')
data.shape

# experiments for manual selection of lines

In [ ]:
# rebound_best_parameters = optimize_parameters(data, rebound_analysis, calculate_accumulated_price_changes,
#                                               windows_size= [i for i in range(1, 366, 1)],
#                                               bias= [i for i in range(1, 50, 1)]
#                                                 )

# print(f"Najlepsze opcje: {rebound_best_parameters}")
rebound_best_parameters = {'window_size': 27, 'bias': 8}

# Najlepsze opcje: {'window_size': 27, 'bias': 8}

In [ ]:
current_data_optimize = define_level(data, rebound_best_parameters["window_size"] ,
                                  rebound_best_parameters["bias"])

rebound_data = rebound_analysis(current_data_optimize)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

In [ ]:
sum(current_score) / (len(data)  /(22 * 12 )) / 12

# Experiments with Genetic Algorithm for line

In [ ]:
# class GeneticAlgorithm:
#     """
#     A class representing a Genetic Algorithm for parameter optimization in trading strategies.

#     Attributes:
#         data (pd.DataFrame): The financial data used for analysis.
#         analysis (callable): The analysis function to process the data.
#         calculate (callable): The scoring function to evaluate the performance of parameter combinations.
#         window_sizes (list): Possible values for the window size parameter.
#         biases (list): Possible values for the bias parameter.
#         best_individual (list): The best parameter combination found by the genetic algorithm.

#     Methods:
#         init_individual(individual_class): Initializes an individual for the genetic algorithm.
#         evaluate(individual): Evaluates the fitness of an individual based on the provided scoring function.
#         mutate(individual): Performs mutation on an individual to introduce diversity in the population.
#         calculate_parameters(params): Calculates the score for a given set of parameters.
#         run_genetic_algorithm(population_size, offspring_size, cx_probability, mut_probability, n_generations):
#             Runs the genetic algorithm to find the best parameter combination.
#         get_best_individual(): Returns the best parameter combination found by the genetic algorithm.
#     """
    
#     def __init__(self, data, analysis, calculate, window_sizes, biases=[1]):
#         """Initializes the GeneticAlgorithm instance.

#         Args:
#             data (pd.DataFrame): The financial data used for analysis.
#             analysis (callable): The analysis function to process the data.
#             calculate (callable): The scoring function to evaluate the performance of parameter combinations.
#             window_sizes (list): Possible values for the window size parameter.
#             biases (list, optional): Possible values for the bias parameter. Defaults to [1].
#         """
#         # Initialize the GeneticAlgorithm instance with the provided parameters
#         # Attribute Initialization:
#         self.data = data
#         self.analysis = analysis
#         self.calculate = calculate
#         self.window_sizes = window_sizes
#         self.biases = biases
#         self.best_individual = None

#         # Problem Definition
#         # Define the problem as a maximization problem
#         creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#         creator.create("Individual", list, fitness=creator.FitnessMax)

#         # Toolbox Initialization
#         # Create a toolbox with the necessary components
#         self.toolbox = base.Toolbox()

#         # Registering Functions in the Toolbox
#         # Register an initialization method for individuals
#         self.toolbox.register("individual", self.init_individual, creator.Individual)
        
#         # Register a method to initialize a population of individuals
#         self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
#         # Register the evaluation method for individuals
#         self.toolbox.register("evaluate", self.evaluate)
        
#         # Register the crossover method using Blend Crossover with a specified alpha value
#         self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        
#         # Register the mutation method
#         self.toolbox.register("mutate", self.mutate)
        
#         # Register the selection method using Tournament Selection with a tournament size of 3
#         self.toolbox.register("select", tools.selTournament, tournsize=3)

    
#     def init_individual(self, individual_class):
#         """
#         Initializes an individual for the genetic algorithm.

#         Args:
#             individual_class: The class representing an individual in the genetic algorithm.

#         Returns:
#             list: The initialized individual.
#         """
        
#         return individual_class([random.choice(self.window_sizes), random.choice(self.biases)])

    
#     def evaluate(self, individual):
#         """
#         Evaluates the fitness of an individual based on the provided scoring function.

#         Args:
#             individual (list): The individual representing a parameter combination.

#         Returns:
#             tuple: The fitness score of the individual.
#         """
        
#         window_size, bias = map(int, individual)
#         params = {'window_size': window_size, 'bias': bias}
#         score = self.calculate_parameters(params)
#         return (score,)

    
#     def mutate(self, individual):
#         """
#         Performs mutation on an individual to introduce diversity in the population.

#         Args:
#             individual (list): The individual to be mutated.

#         Returns:
#             tuple: The mutated individual.
#         """
        
#         if random.random() < 0.5:
#             individual[0] = int(abs(individual[0] + random.randint(-5, 5)))
#         else:
#             individual[1] = random.choice(self.biases)
#         return individual,

    
#     def calculate_parameters(self, params):
#         """
#         Calculates the score for a given set of parameters.

#         Args:
#             params (dict): The parameter values.

#         Returns:
#             float: The calculated score.
#         """
        
#         current_data = define_level(self.data, params['window_size'], params['bias'])
#         rebound_data = self.analysis(current_data)
#         return self.calculate(rebound_data)

        
#     def run_genetic_algorithm(self, population_size=10, offspring_size=50, cx_probability=0.7, mut_probability=0.2, n_generations=10):
#         """
#         Runs the genetic algorithm to find the best parameter combination.

#         Args:
#             population_size (int, optional): The size of the initial population. Defaults to 10.
#             offspring_size (int, optional): The size of the offspring population. Defaults to 50.
#             cx_probability (float, optional): The crossover probability. Defaults to 0.7.
#             mut_probability (float, optional): The mutation probability. Defaults to 0.2.
#             n_generations (int, optional): The number of generations. Defaults to 10.

#         Returns:
#             list: The best parameter combination found by the genetic algorithm.
#         """
#         # Create an initial population
#         population = self.toolbox.population(n=population_size)

#         # Run the genetic algorithm
#         algorithms.eaMuPlusLambda(population, self.toolbox, mu=population_size, lambda_=offspring_size,
#                                   cxpb=cx_probability, mutpb=mut_probability, ngen=n_generations, stats=None, halloffame=None)

#         # Get the best individual
#         self.best_individual = tools.selBest(population, k=1)[0]
#         print("Best Parameters:", self.best_individual)

#         return self.best_individual

    
#     def get_best_individual(self):
#         """
#         Returns the best parameter combination found by the genetic algorithm.

#         Returns:
#             list: The best parameter combination.
#         """
#         return self.best_individual


# # Example usage of the GeneticAlgorithm class
# # Create an instance of the class
# genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
#                                      window_sizes= [i for i in range(1, 366, 1)], 
#                                      biases= [i for i in range(1, 50, 1)])


# # Run the genetic algorithm
# best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
#                                                           offspring_size=100, 
#                                                           cx_probability=0.7, 
#                                                           mut_probability=0.2, 
#                                                           n_generations=50)

In [ ]:
# # Create an instance of the class
# genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
#                                      window_sizes= [i for i in range(1, 366, 1)], 
#                                      biases= [i for i in range(1, 50, 1)])


# # Run the genetic algorithm
# best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
#                                                           offspring_size=100, 
#                                                           cx_probability=0.7, 
#                                                           mut_probability=0.2, 
#                                                           n_generations=50)

In [ ]:
# current_data_algorithm = define_level(data,
#                 ceil(genetic_algorithm.get_best_individual()[0]),
#                 ceil(genetic_algorithm.get_best_individual()[1]) )

# rebound_data = rebound_analysis(current_data_algorithm)
# current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
# current_score

In [ ]:
# sum(current_score) / (data(data)  /(22 * 12 )) / 12


# Order

In [ ]:
# automated_trading_from_signals(rebound_data, symbol= symbol)

# data preprocessing with class Preprocessing_stock_data


In [ ]:
del data
data = get_historical_data() # D1

In [ ]:
columns_name = {'Open':'open', 'High' : 'high' , 'Low' : 'low', 
                'Close' : 'close', "Volume":"volume" }
data = data.rename(columns= columns_name)
data.sample(5)

In [ ]:
preprocessor = Preprocessing_stock_data(data)

data_indicators_pattern  =  preprocessor.add_indicators_pattern_recognition_functions()
data_calculate_overlap_studies  =  preprocessor.calculate_overlap_studies()
data_math_operator_functions  =  preprocessor.math_operator_functions()
data_math_transform_functions  =  preprocessor.math_transform_functions()
data_momentum_indicator_functions  =  preprocessor.momentum_indicator_functions()
data_statistic =  preprocessor.statistic_functions()


In [ ]:
print(data_indicators_pattern.shape)
print(data_calculate_overlap_studies.shape)
print(data_math_operator_functions.shape)
print(data_math_operator_functions.shape)
print(data_math_transform_functions.shape)
print(data_momentum_indicator_functions.shape)
print(data_statistic.shape)

In [ ]:
data_math_transform_functions

# experiments with RFELSTM and data from Preprocessing_stock_data

In [ ]:
data_indicators_pattern.sample(5)

## experiments № 1 Attention layers

In [ ]:
%%time
class AttentionRFE:
    def __init__(self, data):
        self.data = data
        self.X = None
        self.y = None
        self.model = None

    def prepare_data(self, window_size=25, y_column="close"):
        X, y = [], []
    
        for i in range(len(self.data) - window_size):
            window = self.data.iloc[i:(i + window_size)].drop(y_column, axis=1)
            X.append(window.values)
            y.append(self.data.loc[i + window_size, y_column])
    
        self.X = np.array(X)
        self.y = np.array(y)
    
        return self.X, self.y

    def model_LSTM(self, shape: list[int, int]):
        try:
            input_layer = Input(shape = shape )
            
            lstm_layer = LSTM(64, activation='relu', return_sequences=True)(input_layer)
            output_tensor, weights = Attention()([lstm_layer, lstm_layer], return_attention_scores=True)
            flatten_layer = Flatten()(output_tensor)
            output_layer = Dense(1, activation='linear',)(flatten_layer)
    
            model = Model(inputs=input_layer, outputs=output_layer)
            # model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
            self.model = model
            return self.model
    
        except AttributeError as e:
            print(e)
            if self.X is None or self.y is None:
                self.prepare_data()
            return self.model_LSTM()

    def fit(self, X=None, y=None, num_features_to_keep=1, window_size=25, epochs=10):
        
        if not X and not y:
            self.prepare_data(window_size)
        else:
            self.X = X
            self.y = y

    
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
    
        for _ in tqdm(range(self.X.shape[2] - num_features_to_keep)):
            
            model = clone_model(self.model_LSTM( shape=[X_train.shape[1], X_train.shape[2]]))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
            # Обучение модели на текущих данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
            # Оценка модели на тестовых данных до обучения
            y_pred_before = model.predict(X_test)
            mse_before = mean_squared_error(y_test, y_pred_before)
    
            # Получение весов из слоя Attention
            attention_layer = model.layers[2]
    
            # Предполагая, что attention_layer - это MultiHeadAttention слой из TensorFlow
            _, attention_scores = attention_layer([X_test[:1], X_train[:1]], return_attention_scores=True)
            print(attention_scores)

    
            # Суммирование весов по признакам
            feature_importance = tf.reduce_sum(attention_scores, axis=2)
            print(feature_importance)
    
    
            # Находим индекс признака с наименьшим весом
            idx_to_remove = np.argmin(feature_importance)
            # print(idx_to_remove)
    
            # Удаляем признак с наименьшим весом из данных
            X_train = np.delete(X_train, idx_to_remove, axis=2)
            X_test = np.delete(X_test, idx_to_remove, axis=2)
            
    
            # Обновление модели
            model = clone_model(self.model_LSTM([X_train.shape[1], X_train.shape[2]] ))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
            
            # Обучение модели на новых данных
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
    
            # Оценка модели на тестовых данных после обучения
            y_pred_after = model.predict(X_test)
            mse_after = mean_squared_error(y_test, y_pred_after)
    
            print(f"Removed feature at index {idx_to_remove}. Test MSE before training: {mse_before}. Test MSE after training: {mse_after}")


## Создание экземпляра класса AttentionRFE
            
rfe = AttentionRFE(data_indicators_pattern)
# rfe.fit()

## experiments GradientRFE № 2

Nasza warstwa Attention nie zachowuje się zbyt dziwnie i nie zwraca oczekiwanych danych.

"Problem z pierwszą częścią polega na tym, że usuwa tylko jedną cechę, a następnie ponownie przeprowadza wyszukiwanie i usuwa inną, i tak dalej do n. Jednak co, jeśli po połączeniu słabej cechy, która została wcześniej usunięta, z tymi, które pozostały po n iteracjach, wynik okazałby się lepszy? Konieczne jest opracowanie metody łączenia parametrów. Jednak implementacja czegoś takiego byłaby bardzo nieoptymalna, ponieważ czas wyszukiwania najlepszych parametrów wyniósłby n!, gdzie n to liczba cech. Spróbujmy podejść do zadania z innej strony"

In [ ]:
%%time

data_indicators_pattern  =  preprocessor.add_indicators_pattern_recognition_functions()
data_calculate_overlap_studies  =  preprocessor.calculate_overlap_studies()
data_math_operator_functions  =  preprocessor.math_operator_functions()
data_math_transform_functions  =  preprocessor.math_transform_functions()
data_momentum_indicator_functions  =  preprocessor.momentum_indicator_functions()
data_statistic =  preprocessor.statistic_functions()


data = [data_indicators_pattern, data_calculate_overlap_studies, data_math_operator_functions,
        data_math_transform_functions, data_momentum_indicator_functions, data_statistic]

data_name = ["data_indicators_pattern", "data_calculate_overlap_studies", "data_math_operator_functions",
             "data_math_transform_functions", "data_momentum_indicator_functions", "data_statistic"]


def process_dataframes(data:[pd.DataFrame], df_name:[str]) -> dict:

    best_parameters = {}
    for name, df in tqdm(zip(df_name, data)):
        df = df.loc[:, df.nunique() != 1]
        grfe = GradientRFE(df)  
        grfe.fit() 
        best_parameters[name] = grfe.get_params()

    return best_parameters


best_parameters = process_dataframes(data, data_name)

In [ ]:
data_indicators_pattern  =  data_indicators_pattern.iloc[:, features_to_indicators_pattern]
data_calculate_overlap_studies  =  data_calculate_overlap_studies.iloc[:, features_to_calculate_overlap_studi]
data_math_operator_functions  =  data_math_operator_functions.iloc[:, features_to_math_operator_functions]
data_math_transform_functions  =  data_math_transform_functions.iloc[:, features_to_math_transform_function]
data_momentum_indicator_functions  =  data_momentum_indicator_functions.iloc[:, features_to_momentum_indicator_func]
data_statistic =  data_statistic.iloc[:, features_to_statistic]

In [ ]:
data_indicators_pattern

In [ ]:
data_statistic.nunique()